# **Denoising: Modern Methods**

## Data Preview (Before Denoising)

In [ ]:
# Same with "Denoising: Traditional Methods"

### Load Extracted Data
#### Note: 2B Train Raw (30000 samples, 32 channels, 256 timesteps)

In [1]:
import pandas as pd
import numpy as np
import os

path = "/kaggle/input/raw-eeg-2b"
X_batch = pd.read_csv(os.path.join(path, "X_raw_train2B.csv"), header=0)
y_batch = pd.read_csv(os.path.join(path, "y_raw_train2B.csv"), header=0)

X_batch = X_batch.to_numpy()
labels = y_batch.to_numpy().flatten()

n_channels = 32
n_timesteps = 256
X = X_batch.reshape(-1, n_channels, n_timesteps)

print(X.shape)
print(labels.shape)

(30000, 32, 256)
(30000,)


In [4]:
from keras.utils import to_categorical

X_train = X.transpose(0, 2, 1)
print(X_train.shape)

y_train = to_categorical(labels, num_classes=10).astype('float32')
print(y_train.shape)

(30000, 256, 32)
(30000, 10)


### Load Preprocessed Data
#### Note: 15460 samples, 32 channels, 125 timesteps

In [ ]:
import os
import pickle

path = '/kaggle/input/preprocessed-final-2/'
with open(os.path.join(path, "processed_eeg_data-2.pkl"), "rb") as f:
    processed = pickle.load(f)

print(processed.keys())
print(processed['X'].shape)
print(processed['y'].shape)

## DENOISING METHODS

### **SDAE (Supervised Denoising Autoencoder)**

#### RAW DATA

In [25]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, TimeDistributed, Bidirectional, Reshape

def build_lstm_sdae(input_shape, latent_dim=64, num_classes=10):
    # Input: (timesteps, channels)
    inputs = Input(shape=input_shape, name='input')

    
    # ENCODER
    x = Bidirectional(LSTM(64, return_sequences=True))(inputs)
    x = Dropout(0.3)(x)
    x = Bidirectional(LSTM(64))(x)
    latent = Dense(latent_dim, activation='relu', name='latent')(x)

    
    # Classifier
    x_cls = Dense(128, activation='relu')(latent)
    x_cls = Dropout(0.3)(x_cls)
    x_cls = Dense(64, activation='relu')(x_cls)
    x_cls = Dropout(0.2)(x_cls)
    class_output = Dense(10, activation='softmax', name='class_output')(x_cls)

    
    # DECODER
    x = Dense(input_shape[0] * 64, activation='relu')(latent)
    x = Dropout(0.3)(x)
    x = Reshape((input_shape[0], 64))(x)
    x = LSTM(64, return_sequences=True)(x)
    x = TimeDistributed(Dense(input_shape[1]), name='reconstruction')(x)

    
    # MODEL
    model = Model(inputs=inputs, outputs=[x, class_output])
    model.compile (
        optimizer    ='adam',
        loss         = {'reconstruction': 'mse', 'class_output': 'categorical_crossentropy'},
        loss_weights = {'reconstruction': 1.0, 'class_output': 10.0},
        metrics      = {'class_output': 'accuracy'}
    )
    return model

In [26]:
model = build_lstm_sdae(input_shape=(256, 32))
model.fit ( 
    X_train, 
    {
        'reconstruction' : X_train, 
        'class_output'   : y_train
    }, 
    epochs           = 50, 
    batch_size       = 256, 
    validation_split = 0.2
)

Epoch 1/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 39s 202ms/step - class_output_accuracy: 0.1008 - class_output_loss: 2.3230 - loss: 402898432.0000 - reconstruction_loss: 402896672.0000 - val_class_output_accuracy: 0.1075 - val_class_output_loss: 2.3025 - val_loss: 472223808.0000 - val_reconstruction_loss: 469352800.0000
Epoch 2/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 14s 153ms/step - class_output_accuracy: 0.1045 - class_output_loss: 2.3062 - loss: 404138656.0000 - reconstruction_loss: 404139744.0000 - val_class_output_accuracy: 0.1153 - val_class_output_loss: 2.3016 - val_loss: 472198016.0000 - val_reconstruction_loss: 469327264.0000
Epoch 3/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 14s 154ms/step - class_output_accuracy: 0.1073 - class_output_loss: 2.3041 - loss: 404960640.0000 - reconstruction_loss: 404960064.0000 - val_class_output_accuracy: 0.1152 - val_class_output_loss: 2.3015 - val_loss: 472174112.0000 - val_reconstruction_loss: 469303520.0000
Epoch 4/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 15s 155ms/step - class_output

#### PREPROCESSED DATA

### **LSTM Autoencoder**
#### Note:
* Input = Raw Data (trimmed)
* Output = Reconstructed Data
* Monitor = Data Cleaned using Traditional Methods

## Data Review (After Denoising)

### SDAE (Preprocessed Data)

### LSTM Autoencoder